In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pickle
import json
from bread import vis
import itertools
import numpy as np
import pandas as pd
from bread.algo import tracking
from bread.data import *
from sklearn.metrics import confusion_matrix
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
from glob import glob
import sys
import os.path
import datetime, json
from bread.algo.tracking import AssignmentDataset, GNNTracker, AssignmentClassifier, GraphLoader, accuracy_assignment, f1_assignment
from bread.data import *
from bread.algo.tracking import AssignmentDataset, GNNTracker, AssignmentClassifier, GraphLoader, accuracy_assignment, f1_assignment

from glob import glob
from pathlib import Path
import datetime, json, argparse
import torch, os
from pprint import pprint
from sklearn.metrics import confusion_matrix


from skorch.dataset import ValidSplit
from skorch.callbacks import LRScheduler, Checkpoint, ProgressBar, EpochScoring, WandbLogger, EarlyStopping

from importlib import import_module

import tqdm
import numpy as np, scipy.optimize, scipy.sparse
import re


Matplotlib is building the font cache; this may take a moment.


In [4]:
data_path = '/mlodata1/hokarami/fari/bread/data'
segmentation_paths =[
    os.path.join(data_path,'YIT/test_set_1_segmentation.h5'),
    os.path.join(data_path,'YIT/test_set_2_segmentation.h5'),
    os.path.join(data_path,'YIT/test_set_3_segmentation.h5'),
    os.path.join(data_path,'YIT/test_set_4_segmentation.h5'),
    os.path.join(data_path,'YIT/test_set_5_segmentation.h5'),
    os.path.join(data_path,'YIT/test_set_6_segmentation.h5'),
    os.path.join(data_path,'YIT/test_set_7_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony000_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony001_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony002_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony003_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony004_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony005_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony006_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony007_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony008_segmentation.h5'),
    
    ]

In [26]:
all_details = pd.DataFrame(columns=['colony', 'frame_num', 'cell_num', 'area', 'ecc', 'cell_number', 'cell_age'])
for seg_file in tqdm.tqdm(segmentation_paths):
    if "colony" in seg_file:
        mic_file = seg_file.replace("segmentations","microscopies").replace("segmentation","microscopy").replace('.h5', '.tif')
    elif "YIT" in seg_file:
        mic_file = seg_file.replace("_segmentation.h5",".tif")
    colony_name_pattern =  r'(\w+)_segmentation'
    colony_name = re.findall(colony_name_pattern, seg_file)[0]
    seg = SegmentationFile.from_h5(seg_file).get_segmentation("FOV0")
    # microscopy = Microscopy.from_tiff(mic_file)
    feat = Features(seg, nn_threshold=12, scale_length=1, scale_time=1)
    # start = min(seg.get_frame_count()-18,18)
    start = 0
    end = min(start+100, seg.get_frame_count())
    # end = seg.get_frame_count()
    for frame in tqdm.tqdm(range(start,end), desc=colony_name, leave=True):
        cells_ids = seg.cell_ids(frame)
        for cell in cells_ids:
            area = feat.cell_area(frame, cell)
            # ecc = feat.cell_ecc(frame, cell)
            ecc = 0
            cell_age = feat.cell_age(frame, cell)
            # cell_age = 0
            row = {'colony':colony_name, 'frame_num':frame, 'cell_num':cell, 'area':area, 'ecc':ecc, 'cell_number':len(cells_ids), 'cell_age': cell_age}
            all_details.loc[len(all_details)] = row
all_details['diametere'] = np.sqrt(all_details['area']/np.pi)*2

            
    
    

  0%|          | 0/6 [00:00<?, ?it/s]/mlodata1/hokarami/fari/bread/src/bread/data/_data.py:623: MultipleContoursWarning: OpenCV returned multiple contours, 2 found.
  warnings.warn(Contour.MultipleContoursWarning(len(contours_cv)))


this happened for cell_id=33, time_id=66


 33%|███▎      | 2/6 [1:55:56<3:13:29, 2902.40s/it]

In [ ]:
all_details

,colony,frame_num,cell_num,area,ecc,cell_number,cell_age
0,colony000,0,1,564.5,0,4,0
1,colony000,0,2,1485.0,0,4,0
2,colony000,0,3,2262.5,0,4,0
3,colony000,0,4,125.5,0,4,0
4,colony000,1,1,664.5,0,4,1
...,...,...,...,...,...,...,...
75,colony000,18,4,1487.0,0,6,18
76,colony000,18,5,58.5,0,6,0
77,colony000,18,6,44.5,0,6,0
78,colony000,19,1,1776.0,0,6,19


45.31927933665922

In [15]:
def calculate_cell_areas_pixels(seg):
    # Extract unique cell labels (excluding background label 0)
    all_cell_areas = np.array([])
    for frame in range(seg.data.shape[0]):
        data = seg.data[frame]
        unique_labels = np.unique(data[data != 0])

        # Calculate the area of each cell
        cell_areas = np.array([np.sum(data == label) for label in unique_labels])
        all_cell_areas = np.concatenate((all_cell_areas, cell_areas))

    # Calculate the average cell size in pixels
    average_cell_size = np.mean(all_cell_areas)

    # Calculate the average cell diameter in pixels (assuming cells are approximately circular)
    return all_cell_areas

In [16]:
all_cell_areas = np.array([])
for seg_file in tqdm.tqdm(segmentation_paths):
    if "colony" in seg_file:
        mic_file = seg_file.replace("segmentations","microscopies").replace("segmentation","microscopy").replace('.h5', '.tif')
    elif "YIT" in seg_file:
        mic_file = seg_file.replace("_segmentation.h5",".tif")
    colony_name_pattern =  r'(\w+)_segmentation'
    colony_name = re.findall(colony_name_pattern, seg_file)[0]
    seg = SegmentationFile.from_h5(seg_file).get_segmentation("FOV0")
    cell_areas = calculate_cell_areas_pixels(seg)
    all_cell_areas = np.concatenate((all_cell_diameters, cell_diameters))
np.qrt(np.mean(all_cell_areas))*2
        
            

100%|██████████| 6/6 [01:42<00:00, 17.14s/it]


1749.2194155156708

In [21]:
np.sqrt(np.mean(all_cell_diameters))*2


83.6473410340262

In [ ]:
all_details['diametere'] = np.sqrt(all_details['area']/np.pi)*2
all_details['diametere'].mean()

In [ ]:
import plotly.express as px


In [34]:
selected_colonies = all_details#.loc[all_details['colony'].isin(['colony008', 'colony006','colony005','colony001', 'colony000', 'test_set_1', 'test_set_4' ])]
selected_colonies=selected_colonies.loc[selected_colonies['cell_age']!=selected_colonies['frame_num']] 
fig = px.scatter(selected_colonies, x="cell_age", y="area", color="colony")
fig.show()



In [29]:
data_path = '/mlodata1/hokarami/fari/bread/data'
segmentation_paths =[
    os.path.join(data_path,'YIT/test_set_1_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_2_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_3_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_4_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_5_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_6_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_7_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony000_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony001_segmentation.h5'),
    # os.path.join(data_path,'segmentations/colony002_segmentation.h5'),
    # os.path.join(data_path,'segmentations/colony003_segmentation.h5'),
    # os.path.join(data_path,'segmentations/colony004_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony005_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony006_segmentation.h5'),
    # os.path.join(data_path,'segmentations/colony007_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony008_segmentation.h5'),
    ]
start = 40
all_details_40_58 = pd.DataFrame(columns=['colony', 'frame_num', 'cell_num', 'area', 'ecc', 'cell_number', 'cell_age'])
for seg_file in tqdm.tqdm(segmentation_paths):
    if "colony" in seg_file:
        mic_file = seg_file.replace("segmentations","microscopies").replace("segmentation","microscopy").replace('.h5', '.tif')
    elif "YIT" in seg_file:
        mic_file = seg_file.replace("_segmentation.h5",".tif")
    colony_name_pattern =  r'(\w+)_segmentation'
    colony_name = re.findall(colony_name_pattern, seg_file)[0]
    seg = SegmentationFile.from_h5(seg_file).get_segmentation("FOV0")
    # microscopy = Microscopy.from_tiff(mic_file)
    feat = Features(seg, nn_threshold=12, scale_length=1, scale_time=1)
    for frame in tqdm.tqdm(range(start,start+18), desc=colony_name):
        cells_ids = seg.cell_ids(frame)
        for cell in cells_ids:
            area = feat.cell_area(frame, cell)
            ecc = feat.cell_ecc(frame, cell)
            cell_age = feat.cell_age(frame, cell)
            row = {'colony':colony_name, 'frame_num':frame, 'cell_num':cell, 'area':area, 'ecc':ecc, 'cell_number':len(cells_ids), 'cell_age': cell_age}
            all_details_40_58.loc[len(all_details_40_58)] = row

    

100%|██████████| 6/6 [12:42<00:00, 127.06s/it]


In [30]:
all_details_40_58

,colony,frame_num,cell_num,area,ecc,cell_number,cell_age
0,test_set_1,40,1,456.0,0.156477,21,40
1,test_set_1,40,2,678.0,0.492280,21,40
2,test_set_1,40,3,863.0,0.239440,21,40
3,test_set_1,40,4,630.5,0.428111,21,40
4,test_set_1,40,5,575.5,0.329993,21,40
...,...,...,...,...,...,...,...
1750,colony008,57,20,1166.5,0.571205,24,8
1751,colony008,57,21,678.5,0.399654,24,3
1752,colony008,57,22,742.5,0.321632,24,3
1753,colony008,57,23,449.0,0.386385,24,2


In [31]:
selected_colonies_40_58=all_details_40_58.loc[all_details_40_58['cell_age']!=all_details_40_58['frame_num']] 
fig = px.scatter(selected_colonies_40_58, x="cell_age", y="area", color="colony")
fig.show()

In [35]:
data_path = '/mlodata1/hokarami/fari/bread/data'
segmentation_paths =[
    os.path.join(data_path,'YIT/test_set_1_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_2_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_3_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_4_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_5_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_6_segmentation.h5'),
    # os.path.join(data_path,'YIT/test_set_7_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony000_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony001_segmentation.h5'),
    # os.path.join(data_path,'segmentations/colony002_segmentation.h5'),
    # os.path.join(data_path,'segmentations/colony003_segmentation.h5'),
    # os.path.join(data_path,'segmentations/colony004_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony005_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony006_segmentation.h5'),
    # os.path.join(data_path,'segmentations/colony007_segmentation.h5'),
    os.path.join(data_path,'segmentations/colony008_segmentation.h5'),
    ]
start = 20
duration = 36
all_details_20_56 = pd.DataFrame(columns=['colony', 'frame_num', 'cell_num', 'area', 'ecc', 'cell_number', 'cell_age'])
for seg_file in tqdm.tqdm(segmentation_paths):
    if "colony" in seg_file:
        mic_file = seg_file.replace("segmentations","microscopies").replace("segmentation","microscopy").replace('.h5', '.tif')
    elif "YIT" in seg_file:
        mic_file = seg_file.replace("_segmentation.h5",".tif")
    colony_name_pattern =  r'(\w+)_segmentation'
    colony_name = re.findall(colony_name_pattern, seg_file)[0]
    seg = SegmentationFile.from_h5(seg_file).get_segmentation("FOV0")
    # microscopy = Microscopy.from_tiff(mic_file)
    feat = Features(seg, nn_threshold=12, scale_length=1, scale_time=1)
    for frame in tqdm.tqdm(range(start,start+duration), desc=colony_name):
        cells_ids = seg.cell_ids(frame)
        for cell in cells_ids:
            area = feat.cell_area(frame, cell)
            ecc = feat.cell_ecc(frame, cell)
            cell_age = feat.cell_age(frame, cell)
            row = {'colony':colony_name, 'frame_num':frame, 'cell_num':cell, 'area':area, 'ecc':ecc, 'cell_number':len(cells_ids), 'cell_age': cell_age}
            all_details_20_56.loc[len(all_details_20_56)] = row

    

100%|██████████| 6/6 [14:24<00:00, 144.08s/it]


In [36]:
all_details_20_56

,colony,frame_num,cell_num,area,ecc,cell_number,cell_age
0,test_set_1,20,1,370.5,0.281108,18,20
1,test_set_1,20,2,624.0,0.408665,18,20
2,test_set_1,20,3,814.0,0.365205,18,20
3,test_set_1,20,4,639.0,0.319628,18,20
4,test_set_1,20,5,544.5,0.346635,18,20
...,...,...,...,...,...,...,...
2717,colony008,55,19,1207.0,0.403473,23,7
2718,colony008,55,20,1006.0,0.627861,23,6
2719,colony008,55,21,525.5,0.438766,23,1
2720,colony008,55,22,390.0,0.542296,23,1


In [37]:
selected_colonies_20_56 = all_details_20_56#.loc[all_details_20_56['colony'].isin(['colony008', 'colony006','colony005','colony001', 'colony000', 'test_set_1', 'test_set_4' ])]
selected_colonies_20_56=selected_colonies_20_56.loc[selected_colonies_20_56['cell_age']!=selected_colonies_20_56['frame_num']] 
fig = px.scatter(selected_colonies_20_56, x="cell_age", y="area", color="colony")
fig.show()

In [40]:
import numpy as np

selected_colonies_20_56 = all_details_20_56.loc[all_details_20_56['cell_age']!=all_details_20_56['frame_num']] 
grouped = selected_colonies_20_56.groupby(['cell_age', 'colony']).median().reset_index()
grouped['sqrt_area'] = np.sqrt(grouped['area'])
fig = px.scatter(grouped, x="cell_age", y="sqrt_area", color="colony")
fig.show()

In [7]:
all_details_20_56.median()

/tmp/ipykernel_856/352375575.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  all_details.median()


frame_num       12.000000
cell_num        71.000000
area           673.250000
ecc              0.526844
cell_number    149.000000
cell_age         8.000000
dtype: float64

In [41]:
all_details.groupby(['colony']).median()

,frame_num,cell_num,area,ecc,cell_number,cell_age
colony,,,,,,
colony000,28.0,5.0,1595.5,0.523928,8.0,15.0
colony001,29.0,3.0,2355.0,0.509935,7.0,20.0
colony002,28.0,6.0,2066.0,0.414554,11.0,21.0
colony003,27.0,2.0,1780.0,0.400797,4.0,18.0
colony004,27.0,2.0,1465.0,0.456080,3.0,23.0
colony005,28.0,3.0,1760.0,0.488031,4.0,22.0
colony006,28.0,5.0,577.0,0.550829,10.0,17.0
colony007,29.0,88.0,1755.0,0.555289,217.0,18.0
colony008,25.0,5.0,1534.5,0.444147,8.0,19.0


In [44]:
all_details.loc[all_details['colony']!='colony006'].median()

/tmp/ipykernel_856/3943442130.py:1: FutureWarning:

The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



frame_num       12.000000
cell_num        71.000000
area           678.500000
ecc              0.525612
cell_number    149.000000
cell_age         8.000000
dtype: float64

In [6]:
import tifffile

In [7]:
def get_calibration_factor(tif_file_path):
    with tifffile.TiffFile(tif_file_path) as tif:
        # Check for tags that may contain calibration information
        tags = tif.pages[0].tags

        # You need to identify the specific tags used by your microscope
        # The tags might be named something like 'XResolution' and 'YResolution'
        x_resolution = tags.get('XResolution', None)
        y_resolution = tags.get('YResolution', None)

        if x_resolution is not None and y_resolution is not None:
            # The resolution tags might contain the calibration information
            x_resolution_value = x_resolution.value[0] / x_resolution.value[1]
            y_resolution_value = y_resolution.value[0] / y_resolution.value[1]

            # Assuming resolution is in micrometers per pixel
            return x_resolution_value, y_resolution_value
        else:
            # If resolution tags are not found, you may need to check other tags
            # or consult the documentation of the microscope to identify the correct tags
            print("Calibration information not found in the specified tags.")
            return None, None



In [26]:
tif_paths = [
    os.path.join(data_path,'YIT/test_set_1.tif'),
    os.path.join(data_path,'YIT/test_set_2.tif'),
    os.path.join(data_path,'YIT/test_set_3.tif'),
    os.path.join(data_path,'YIT/test_set_4.tif'),
    os.path.join(data_path,'YIT/test_set_5.tif'),
    os.path.join(data_path,'YIT/test_set_6.tif'),
    os.path.join(data_path,'YIT/test_set_7.tif'),
    os.path.join(data_path,'microscopies/colony000_microscopy.tif'),
    os.path.join(data_path,'microscopies/colony001_microscopy.tif'),
    os.path.join(data_path,'microscopies/colony002_microscopy.tif'),
    os.path.join(data_path,'microscopies/colony003_microscopy.tif'),
    os.path.join(data_path,'microscopies/colony004_microscopy.tif'),
    os.path.join(data_path,'microscopies/colony005_microscopy.tif'),
    os.path.join(data_path,'microscopies/colony006_microscopy.tif'),
    os.path.join(data_path,'microscopies/colony007_microscopy.tif'),
    os.path.join(data_path,'microscopies/colony008_microscopy.tif'),
]

In [27]:
calib_data = pd.DataFrame(columns=['colony', 'x_calibration', 'y_calibration'])
for i, seg_file in enumerate(segmentation_paths):
    colony_name_pattern =  r'(\w+)_segmentation'
    colony_name = re.findall(colony_name_pattern, seg_file)[0]
    tif_file_path = tif_paths[i]
    x_calibration, y_calibration = get_calibration_factor(tif_file_path)
    row = {'colony': colony_name, 'x_calibration': x_calibration, 'y_calibration': y_calibration}
    calib_data = calib_data.append(row, ignore_index=True)

/tmp/ipykernel_1012/1160170350.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calib_data = calib_data.append(row, ignore_index=True)
/tmp/ipykernel_1012/1160170350.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calib_data = calib_data.append(row, ignore_index=True)
/tmp/ipykernel_1012/1160170350.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calib_data = calib_data.append(row, ignore_index=True)
/tmp/ipykernel_1012/1160170350.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calib_data = calib_data.append(row, ignore_index=True)
/tmp/ipykernel_1012/1160170350.py:8: FutureWarning: The frame.append method is deprecated an

In [24]:
x = 1.0/0.1111111
x

9.00000090000009

In [21]:
calib_data

,colony,x_calibration,y_calibration
0,test_set_1,1.000000,1.000000
1,test_set_2,1.000000,1.000000
2,test_set_3,1.000000,1.000000
3,test_set_4,1.000000,1.000000
4,test_set_5,1.000000,1.000000
5,test_set_6,1.000000,1.000000
6,test_set_7,1.000000,1.000000
7,colony000,1.000000,1.000000
8,colony001,9.230769,9.230769
9,colony002,9.230769,9.230769
